In [1]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("trongnguyen04/nckh-2425-crops")
print("Path to dataset files:", path)

# ─── copy raw folder “1” → /content/dataset_original/crops ───────
SRC="/root/.cache/kagglehub/datasets/trongnguyen04/nckh-2425-crops/versions/1"
DEST="/content/dataset_original/crops"
!mkdir -p $(dirname $DEST)
!cp -r "$SRC" "$DEST"

100%|██████████| 107M/107M [00:03<00:00, 34.3MB/s]

Extracting files...


Path to dataset files: /root/.cache/kagglehub/datasets/trongnguyen04/nckh-2425-crops/versions/1


In [2]:
!git clone https://github.com/htrnguyen/compare_ocr_benchmark.git

Cloning into 'compare_ocr_benchmark'...
remote: Enumerating objects: 20, done.
remote: Counting objects: 100% (20/20), done.
remote: Compressing objects: 100% (15/15), done.
remote: Total 20 (delta 3), reused 19 (delta 2), pack-reused 0 (from 0)
Receiving objects: 100% (20/20), 72.75 KiB | 3.83 MiB/s, done.
Resolving deltas: 100% (3/3), done.


In [3]:
!pip install python-Levenshtein jiwer

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 161.7/161.7 kB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 53.4 MB/s eta 0:00:00


In [5]:
import sys
sys.path.append('/content/compare_ocr_benchmark/common')

from image_preprocessing import preprocess_image
from ocr_base import OCRModelBase
from metrics import compute_metrics
from utils import read_annotations, save_results

import os
import pandas as pd
import time

In [6]:
!pip install transformers==4.37.2 timm einops

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.4/129.4 kB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.4/8.4 MB 75.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 19.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 120.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 26.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 55.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 14.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 8.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21

In [7]:
from transformers import AutoModel, AutoTokenizer
import torch
import torchvision.transforms as T
from PIL import Image

In [8]:
class VinternOCRModel(OCRModelBase):
    """
    Class VinternOCRModel cho phép tùy biến prompt theo từng label.
    Tự động chọn prompt theo từng nhãn cho inference tối ưu.
    """
    def __init__(self, model_name="5CD-AI/Vintern-1B-v3_5", device="cuda"):
        super().__init__()
        from transformers import AutoModel, AutoTokenizer
        import torch
        import torchvision.transforms as T

        self.tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
        self.model = AutoModel.from_pretrained(
            model_name,
            torch_dtype=torch.bfloat16,
            low_cpu_mem_usage=True,
            trust_remote_code=True
        ).eval().to(device)
        self.device = device
        self.transform = self.build_transform(448)
        self.prompts = {
            "brand":
                "Trích xuất chính xác TÊN THƯƠNG HIỆU (brand) duy nhất xuất hiện trên ảnh. "
                "Chỉ trả về một chuỗi ký tự duy nhất (brand). Đáp án định dạng JSON: {\"text\": \"...\"}. "
                "Nếu không nhận diện được, trả về: {\"text\": \"None\"}. Tuyệt đối không giải thích gì thêm.",
            "name":
                "Trích xuất chính xác TÊN SẢN PHẨM (name) duy nhất xuất hiện trên ảnh. "
                "Chỉ trả về một chuỗi ký tự duy nhất (name). Đáp án định dạng JSON: {\"text\": \"...\"}. "
                "Nếu không nhận diện được, trả về: {\"text\": \"None\"}. Tuyệt đối không giải thích gì thêm.",
            "date":
                "Trích xuất chính xác NGÀY THÁNG (date) xuất hiện trên ảnh (ưu tiên dạng dd/mm/yyyy hoặc yyyy-mm-dd). "
                "Chỉ trả về một chuỗi ký tự duy nhất (date). Đáp án định dạng JSON: {\"text\": \"...\"}. "
                "Nếu không nhận diện được, trả về: {\"text\": \"None\"}. Tuyệt đối không giải thích gì thêm.",
            "weight":
                "Trích xuất chính xác KHỐI LƯỢNG (weight, bao gồm cả số và đơn vị: g, kg, ml...) xuất hiện trên ảnh. "
                "Chỉ trả về một chuỗi ký tự duy nhất (weight). Đáp án định dạng JSON: {\"text\": \"...\"}. "
                "Nếu không nhận diện được, trả về: {\"text\": \"None\"}. Tuyệt đối không giải thích gì thêm.",
            "default":
                "Trích xuất toàn bộ văn bản (bao gồm cả chữ và số) xuất hiện trong ảnh này. "
                "Chỉ trả về kết quả ở định dạng JSON với khóa 'text'. "
                "Nếu không nhận diện được văn bản nào, trả về: {\"text\": \"None\"}. "
                "Tuyệt đối không giải thích, không bổ sung bất cứ thông tin nào ngoài JSON."
        }

    def build_transform(self, input_size=448):
        import torchvision.transforms as T
        IMAGENET_MEAN = (0.485, 0.456, 0.406)
        IMAGENET_STD = (0.229, 0.224, 0.225)
        return T.Compose([
            T.Lambda(lambda img: img.convert('RGB') if img.mode != 'RGB' else img),
            T.Resize((input_size, input_size), interpolation=T.InterpolationMode.BICUBIC),
            T.ToTensor(),
            T.Normalize(mean=IMAGENET_MEAN, std=IMAGENET_STD)
        ])

    def extract_text_from_json(self, response_text):
        import json
        try:
            start = response_text.find('{')
            end = response_text.rfind('}') + 1
            json_str = response_text[start:end]
            out = json.loads(json_str)
            return out.get("text", "None")
        except Exception:
            return "None"

    def get_prompt(self, label):
        """
        Lấy prompt phù hợp với label.
        Nếu label không thuộc tập đã định nghĩa thì dùng prompt default.
        """
        if label is None or label not in self.prompts:
            return self.prompts["default"]
        return self.prompts[label]

    def predict(self, img_pil, img_np=None, label=None):
        """
        img_pil: ảnh PIL đã preprocess.
        label: nhãn để chọn prompt phù hợp.
        """
        import torch, time
        pixel_values = self.transform(img_pil).unsqueeze(0).to(torch.bfloat16).to(self.device)
        prompt = self.get_prompt(label)
        t1 = time.perf_counter()
        with torch.no_grad():
            response = self.model.chat(
                self.tokenizer,
                pixel_values,
                prompt,
                dict(
                    max_new_tokens=256,
                    do_sample=False,
                    num_beams=3,
                    repetition_penalty=3.5
                )
            )
        t2 = time.perf_counter()
        pred = self.extract_text_from_json(response.strip())
        return pred, t2 - t1

In [9]:
CSV_ANN = '/content/dataset_original/crops/crops_gt.csv'
IMG_DIR = '/content/dataset_original/crops'

df = read_annotations(CSV_ANN)

In [10]:
model = VinternOCRModel()

results = []
for idx, row in df.iterrows():
    fname = row['filename']
    desc_gt = row['description_gt']
    label = row.get('label', '')
    img_path = os.path.join(IMG_DIR, fname)

    img_pil, img_np = preprocess_image(img_path, target_size=448, do_auto=True)
    pred, infer_time = model.predict(img_pil, img_np)
    metrics = compute_metrics(desc_gt, pred)

    results.append({
        "filename": fname,
        "label": label,
        "ground_truth": desc_gt,
        "predicted_text": pred,
        "cer": metrics["cer"],
        "wer": metrics["wer"],
        "lev": metrics["lev"],
        "acc": metrics["acc"],
        "time": round(infer_time, 3)
    })

/usr/local/lib/python3.11/dist-packages/huggingface_hub/file_download.py:943: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

added_tokens.json:   0%|          | 0.00/790 [00:00<?, ?B/s]

...

/usr/local/lib/python3.11/dist-packages/timm/models/layers/__init__.py:48: FutureWarning: Importing from timm.models.layers is deprecated, please import via timm.layers
  warnings.warn(f"Importing from {__name__} is deprecated, please import via timm.layers", FutureWarning)
/usr/local/lib/python3.11/dist-packages/huggingface_hub/file_download.py:943: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


model.safetensors:   0%|          | 0.00/3.75G [00:00<?, ?B/s]

/usr/local/lib/python3.11/dist-packages/huggingface_hub/file_download.py:943: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


generation_config.json:   0%|          | 0.00/129 [00:00<?, ?B/s]

Setting `pad_token_id` to `eos_token_id`:151645 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:151645 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:151645 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:151645 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:151645 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:151645 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:151645 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:151645 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:151645 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:151645 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:151645 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:151645 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:151645 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:151645 for

In [11]:
import os
OUT_CSV = '/content/compare_ocr_benchmark/results/vintern_results.csv'
os.makedirs(os.path.dirname(OUT_CSV), exist_ok=True)
save_results(results, OUT_CSV)

Lưu thành công: /content/compare_ocr_benchmark/results/vintern_results.csv
